# AL Documentation

## Introduction

AL simplifies the complexity of large language models into a simple, open-source solution. One intuitive interface. Multiple AI powerhouses. Ollama, Anthropic, OpenAI - seamlessly integrated into all python environments, you can use it even on 1970 terminals if youd like that sort of a thing.
AL doesn't just fetch answers; it preserves your learning journey. Every interaction, every spark of inspiration, is captured in Jupyter notebooks. Share, version, and automate your AI workflows with ease.

But here's the game-changer: these saved sessions aren't mere logs. They're goldmines for AI advancement. Use them to fine-tune models, iterate rapidly, and fuel your learning. Reuse, edit, and evolve your AI interactions effortlessly.
* simple ASCII/md interface
* model selection
* save/load history as .ipynb file

## Why Use AL?

AL offers several advantages over other similar tools:

1. **Unified Interface**: AL provides a consistent interface for interacting with different AI models, simplifying the process of switching between or comparing various AI services.

2. **Extensibility**: The base class is designed to be easily extended, allowing developers to add support for new AI services by creating subclasses.

3. **History Management**: AL includes built-in conversation history management, making it easy to maintain context in multi-turn conversations.

4. **Code Handling**: The class includes methods for managing and setting code snippets to the clipboard, which is particularly useful for programming-related tasks.

5. **Jupyter Notebook Integration**: AL supports saving and loading conversation history in Jupyter Notebook format, facilitating easy documentation and sharing of AI interactions.

## AL Class Overview

1. The `AL` class serves as the foundation for AI model interactions. Here's an overview of its main components:

```python
class AL:
    def __init__(self, interface: Literal["ollama", "claude", "chatgpt"] = "ollama", model: str = "llama3.1", api_key: Optional[str] = None):
    # implementation

    def ask(self, question):
    # implementation
```

2. To acess the code snipets from the answer:
```python
    def copy_code(def copy_code(self, index: Optional[int] = None):
        # If no index then last code snippet is copied, otherwise the index runs from last to first
```

3. To store/edit as jupyter lab workbook
```python
        
    def save_history(self, filename):
        # Implementation for saving history to a Jupyter Notebook

    def load_history(self, filename):
        # Implementation for loading history from a Jupyter Notebook
```

To add support for a new AI service, create a new subclass of AL and implement the `query` method. Here's a template for creating a new AI service subclass:

```python
class NewAIServiceAL(AL):
    def __init__(self, model, api_key):
        super().__init__(model)
        self.api_key = api_key
        # Add any other necessary initialization

    def ask(self, question):
        # Implement the API call to the new AI service
        # Process the response
        # Update self.history
        # Print the processed response
```

## Conclusion

AL provides a powerful and flexible foundation for interacting with various AI language models. By using AL, developers can easily integrate different AI services into their projects, manage conversation history, and handle code snippets efficiently. The extensible nature of AL allows for easy addition of new AI services, making it a valuable tool for developers working with multiple AI platforms.

## Generic AL object definition

In [110]:
from typing import Literal, Optional, List, Dict
from IPython.display import Markdown
from datetime import datetime
import json
import nbformat as nbf
import pyperclip

class AL:
    def __init__(
        self, 
        interface: Literal["ollama", "claude", "chatgpt"] = "ollama",
        model: str = "llama3.1",
        api_key: Optional[str] = None
    ):
        self.interface = interface
        self.model = model
        self.api_key = api_key
        self.history = []
        self.last_code = []
        self.active_code = 0
        
        # Initialize appropriate client
        self._init_client()

    def _init_client(self):
        if self.interface == "ollama":
            import ollama
            self.client = ollama
        elif self.interface == "claude":
            from anthropic import Anthropic
            self.client = Anthropic(api_key=self.api_key)
        else:  # chatgpt
            import openai
            self.client = openai.OpenAI(api_key=self.api_key)

    def ask(self, message: str):
        # Store user message
        self.history.append({
            "role": "user",
            "content": message,
            "timestamp": datetime.now().isoformat()
        })
        response = self._get_response(self.history)

        # Process response and extract code blocks
        parts = response.split('```')
        for i, part in enumerate(parts):
            if i % 2 == 1:  # Code block
                lines = part.splitlines()
                if lines:
                    code = '\n'.join(lines[1:]).strip()  # Skip language line
                    self.last_code.insert(0, code)  # Add to front of list
        
        # Process and store response
        self.history.append({
            "role": "assistant",
            "content": response,
            "timestamp": datetime.now().isoformat()
        })
        
        # Display response
        display(Markdown(response))

    def _get_response(self, message: list) -> str:
        if self.interface == "ollama":
            response = self.client.chat(
                model=self.model, 
                messages=message
            )
            return response['message']['content']
        elif self.interface == "claude":
            response = self.client.messages.create(
                model=self.model,
                messages=message
            )
            return response.content[0].text
        else:  # chatgpt
            response = self.client.chat.completions.create(
                model=self.model,
                messages=message
            )
            return response.choices[0].message.content

    def copy_code(self, index: Optional[int] = None):
        """Copy specific code block to clipboard"""
        if index is None:
            index = self.active_code
        if 0 <= index < len(self.last_code):
            pyperclip.copy(self.last_code[index])
            print(f"Copied code block {index} to clipboard")
            print(self.last_code[index])
        else:
            print("Invalid code block index")

In [113]:
class AL(AL):
    def split_response(self, response: str) -> List[Dict]:
        """Parse response into markdown and code segments"""
        segments = []
        parts = response.split('```')
        
        for i, part in enumerate(parts):
            if i % 2 == 0:  # Markdown content
                if part.strip():
                    segments.append({
                        "type": "markdown", 
                        "content": part.strip()
                    })
            else:  # Code content
                lines = part.splitlines()
                if lines:
                    language = lines[0].strip()
                    code = '\n'.join(lines[1:]).strip()
                    segments.append({
                        "type": "code", 
                        "language": language, 
                        "content": code
                    })
        
        return segments
    
    def save_history(self, filename: str):
        """Save conversation to notebook"""
        if not filename.endswith('.ipynb'):
            filename += '.ipynb'
            
        nb = nbf.v4.new_notebook()
        cells = []

        for entry in self.history:
            # Add role cell
            cells.append(nbf.v4.new_markdown_cell(
                f"## {entry['role'].capitalize()}"
            ))

            # Split and add content cells
            segments = self.split_response(entry['content'])
            for segment in segments:
                if segment['type'] == 'markdown':
                    cells.append(nbf.v4.new_markdown_cell(
                        segment['content']
                    ))
                else:
                    cells.append(nbf.v4.new_code_cell(
                        segment['content']
                    ))

        nb['cells'] = cells
        
        with open(filename, 'w', encoding='utf-8') as f:
            nbf.write(nb, f)

    def load_history(self, filename: str):
        """Load conversation from notebook"""
        if not filename.endswith('.ipynb'):
            filename += '.ipynb'
            
        with open(filename, 'r', encoding='utf-8') as f:
            nb = nbf.read(f, as_version=4)
        
        self.history = []
        self.last_code = []
        
        current_entry = None
        
        for cell in nb.cells:
            source = cell['source']
            
            if cell['cell_type'] == 'markdown' and source.startswith('## '):
                # If we have a previous entry, add it
                if current_entry:
                    self.history.append(current_entry)
                
                # Start new entry
                role = source[3:].lower().strip()
                current_entry = {
                    'role': role,
                    'content': '',
                    'timestamp': datetime.now().isoformat()
                }
            elif current_entry:
                # Add content to current entry
                if cell['cell_type'] == 'code':
                    if current_entry['role'] == 'assistant':
                        self.last_code.insert(0, source)  # Add to front of list
                    current_entry['content'] += f'```python\n{source}\n```\n'
                else:
                    current_entry['content'] += f'{source}\n'
        
        # Add last entry if exists
        if current_entry:
            self.history.append(current_entry)

In [114]:
chat = AL()

In [115]:
chat.ask("can you write simple random number generator in python")

Here's a simple implementation of a Random Number Generator (RNG) using the `random` module in Python:

```python
import random

class SimpleRandomNumberGenerator:
    def __init__(self, seed=None):
        self.rng = random.Random(seed)

    def generate_number(self, min_value=1, max_value=100):
        return self.rng.randint(min_value, max_value)


# Example usage:
if __name__ == "__main__":
    rng = SimpleRandomNumberGenerator()

    for _ in range(5):
        print(rng.generate_number())
```

In this code:

*   We create a class `SimpleRandomNumberGenerator` to encapsulate the RNG functionality.
*   The `seed` parameter is used to initialize the random number generator with a specific seed value. If no seed is provided, the system clock will be used as the default seed.
*   The `generate_number` method uses the `randint` function from the `random` module to generate a random integer within the specified range (`min_value` and `max_value`). By default, it generates numbers between 1 and 100.

You can adjust the `min_value` and `max_value` parameters when calling `generate_number` to get random numbers in your desired range.

In [103]:
chat.ask("can you make in the same language simple and clever hello world program as well?")

Here's a simple yet clever "Hello World" program in Python:

```python
print("Hello, "+str(ord('W'))+"orld!")
```

This program uses the built-in `ord` function to get the ASCII value of the character 'W' (which is 87), converts it to a string using `str`, and then concatenates it with "Hello, " and "orld!" to print out "Hello, 87orld!".

However, this can be considered clever only in the sense that it uses an ASCII code instead of typing out the word. Here's another example of a clever Hello World program:

```python
def hello():
    return(" "*3)+"Hello,"+(" "*7)+("World!" if input() else "")

hello()
```

This program takes an input from the user (which can be any character). If the input is not empty, it prints out "Hello, World!". Otherwise, it doesn't print anything.

Another example of a clever Hello World program uses bitwise operations to create the string:

```python
print("Hello,"+chr(72)+chr(101)+chr(108)+chr(108)+chr(111)+", "+chr(87)+"orld!")
```

This program converts each character in "Hello, World!" into its ASCII value and then prints out those values as characters.

Finally, here's an example of a clever Hello World program that uses recursion:

```python
def hello(x=""):
    if len(x) < 5:
        print("H" + x)
        return hello("ello,"+x)
    else:
        return

hello()
```

This program recursively prints out the string "Hello, " with increasing length until it reaches 11 characters.

In [107]:
chat.copy_code(1)

Copied code block 1 to clipboard
print("Hello,"+chr(72)+chr(101)+chr(108)+chr(108)+chr(111)+", "+chr(87)+"orld!")


In [108]:
print("Hello,"+chr(72)+chr(101)+chr(108)+chr(108)+chr(111)+", "+chr(87)+"orld!")

Hello,Hello, World!


In [116]:
chat.save_history("test")

In [83]:
chat.last_code

['def greet(func):\n    def wrapper():\n        print("Hello,")\n        func()\n        print("World!")\n    return wrapper\n\n@greet\ndef say_hello():\n    print("I am happy to meet you.")\n\nsay_hello()',
 'print((lambda x: x[0])(["Hello, World!"]))',
 'def hello_world(n=0):\n    print("Hello," * n + " World!" if n == 0 else "World!")\n    hello_world(n+1)\n\nhello_world()',
 'print("Hello, World!")',
 'import random\n\ndef generate_random_number(min_val, max_val):\n    """\n    Generate a random integer between min_val and max_val (inclusive).\n    \n    Args:\n        min_val (int): The minimum value.\n        max_val (int): The maximum value.\n        \n    Returns:\n        int: A random integer between min_val and max_val.\n    """\n    return random.randint(min_val, max_val)\n\n# Example usage\nrandom_number = generate_random_number(1, 100)\nprint(f"Random number between 1 and 100: {random_number}")']

In [91]:
chat.load_history("test.ipynb")

In [92]:
chat.history

[{'role': 'user',
  'content': 'can you write simple random number generator in python\n',
  'timestamp': '2024-12-19T21:41:57.996821'},
 {'role': 'assistant',
  'content': 'Here\'s a simple example of a random number generator using Python\'s built-in `random` module:\n```python\nimport random\n\ndef generate_random_number(min_val, max_val):\n    """\n    Generate a random integer between min_val and max_val (inclusive).\n    \n    Args:\n        min_val (int): The minimum value.\n        max_val (int): The maximum value.\n        \n    Returns:\n        int: A random integer between min_val and max_val.\n    """\n    return random.randint(min_val, max_val)\n\n# Example usage\nrandom_number = generate_random_number(1, 100)\nprint(f"Random number between 1 and 100: {random_number}")\n```\nThis script defines a function `generate_random_number` that takes two integers as arguments, `min_val` and `max_val`, which represent the minimum and maximum values for the random number to be genera